<h1><center>Kadison Singer Experiments 2</center></h1>


### Contents

1. [First Operator Representation](#op1)
    1. [Operator acting on different polynomial](#stabpres1)
    2. [Partially apply operator](#partially)
2. [Second Operator Representation](#op2)
    1. [Not equal to Diagonalized Operator](#diag)
    2. [Final Polynomial is not real stable](#realstable2)
    3. [Equvialence of real rootedness](#equiv)
3. [Third Operator Representation](#op3)
    1. [Quarter Plane Stability](#quatstab)
    
    

In [1]:
include("../polyfunctions.jl")

poly_operator_poly (generic function with 1 method)

## First operator representation <a name="op1"></a>

$$\prod_i^m(1 - \partial_{z_i}\partial_{y_i} + 0.5 \partial_{z_i}^2\partial_{y_i})det(x + \sum_i z_i v_i)*det(x - \sum_i z_i v_i) g_{\mu}(\mathbb{1} + y)$$

In [2]:
function op1(p, i)
    term1 = mvp.differentiate(p,y[i],1)
    term1 = mvp.differentiate(term1,z[i],1)
    term2 = mvp.differentiate(p,y[i],1)
    term2 = mvp.differentiate(term2,z[i],2)
    output = (p - term1 + 0.5*term2)(z[i]=>0, y[i]=>1)
    return output
end

op1 (generic function with 1 method)

### Operator acting on different polynomial <a name="stabpres1"></a>

We investigate what happens when our first operator acts on a different polynomial:

$$det(x + \sum_i z_i v_i)*det(x + \sum_i z_i v_i) g_{\mu}(\mathbb{1} + y)$$

instead of

$$det(x + \sum_i z_i v_i)*det(x - \sum_i z_i v_i) g_{\mu}(\mathbb{1} + y)$$

and we see we quickly get complex roots.

In [3]:
n = 4
m = 3

conj = true
iter = 0
tol = 1e-8

@polyvar x y[1:3] z[1:3]
g = y[1]*y[2] + y[2]*y[3] + y[1]*y[3]
g = 1/3 * g

while conj && iter < 100
    U = randn(n,m)
    matbase = x*eye(n)
    for i = 1:m
       matbase += z[i]*U[:,i]*U[:,i]'
    end

    matbase_prime = x*eye(n)
    for i = 1:m
       matbase_prime += z[i]*U[:,i]*U[:,i]'
    end
    p1 = det(matbase)*det(matbase_prime)*g
    for i = 1:3
        p1 = op1(p1, i)
    end
    p1_roots = mpRoots(p1)
    if maximum(abs.(imag.(p1_roots))) > tol
        @show p1_roots
        conj = false 
    end
    iter +=1
end

@show conj
@show iter

p1_roots = ComplexF64[0.3411501758417052 - 0.14562985630167924im, 0.3411501758417052 + 0.1456298563016793im, 4.895388097608373 + 1.304557849796371im, 4.8953880976083735 - 1.304557849796371im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im]
conj = false
iter = 1


1

### Partially apply operator<a name="partially"></a>

We investigate what happens when apply the operator only partially, meaning for $k \in [n]$ is

$$\prod_i^k(1 - \partial_{z_i}\partial_{y_i} + 0.5 \partial_{z_i}^2\partial_{y_i})det(x + \sum_i z_i v_i)*det(x - \sum_i z_i v_i) g_{\mu}(\mathbb{1} + y) \Bigg|_{z=0=y}$$

real rooted?

In [4]:
n = 4
m = 3

conj = true
iter = 0
tol = 1e-8

@polyvar y[1:3]
g = y[1]*y[2] + y[2]*y[3] + y[1]*y[3]
g = 1/3 * g

while conj && iter < 1000
    U = randn(n,m)
    matbase = x*eye(n)
    for i = 1:m
       matbase += z[i]*U[:,i]*U[:,i]'
    end
    matbase_prime = x*eye(n)
    for i = 1:m
       matbase_prime -= z[i]*U[:,i]*U[:,i]'
    end
    p2= det(matbase)*det(matbase_prime)*g
    for i = 1:2
        p2 = op1(p2, i)
    end
    p2 = p2(z[3]=>0, y[3]=>1)
    p2_roots = mpRoots(p2)
    if maximum(abs.(imag.(p2_roots))) > tol
        @show p2_roots
        conj = false 
    end
    iter +=1
end

@show conj
@show iter

conj = true
iter = 1000


1000

## Second operator representation <a name="op2"></a>

$$ g_{\mu}((1 + \partial_z)(1 - \partial_y))\Bigg|_{z=0=y} \det(xI + \sum_i z_i v_i v_i^\top)\det(xI + \sum_i y_i v_i v_i^\top) $$

### Not equal to Diagonalized Operator<a name="diag"></a>

We want to find out if for $A$ a psd matrix 

$$\det(A + diag((1+ \partial_z)(1 - \partial_y))) \det(xI + V diag(z) V^T)\det(xI + Vdiag(y)V^T)$$

is equal to 

$$\det(D + Odiag((1+ \partial_z)(1 - \partial_y))O^T) \det(xI + VO diag(z) O^TV^T)\det(xI + VOdiag(y)O^TV^T)$$ 

(up to multiplicative constants) for $A = O^TDO$ with $D$ the diagonal matrix containing the eigenvalues of A

we see that the roots differ --> the polynomials are not equivalent

For the second operator representation we make use of the function poly_operator_poly() which we defined in polyfunctions.jl as we need it in both Jupyter notebooks for the Kadison Singer Experiments

In [22]:
# set parameters
m = 3
n = 2
tol2 = 1e-5

@polyvar x z[1:3] y[1:3]  # second parameter must be equal to m!
@polyvar t

# this needs to be mxm dimensional
Diag = Diagonal([y[1] - z[1] - y[1]*z[1], y[2] - z[2] - y[2]*z[2], y[3] - z[3] - y[3]*z[3]]) 
a = [y[1], y[2], y[3], z[1], z[2], z[3]]


# first we create the matrix A
A = rand(1:10,(m,m))
B = A*A'
d = maximum(abs.(eigvals(B)))
A = B /d
F = eigen(A)
eigens = F.values
O = F.vectors

# then we create the polynomial we let the diagonal operator act on 
V = rand(1:10,(n,m))
U = V*O
matbase = x*eye(n)
for i = 1:m
   matbase += z[i]*U[:,i]*U[:,i]'
end
matbase_prime = x*eye(n)
for i = 1:m
   matbase_prime += y[i]*U[:,i]*U[:,i]'
end
p1 = det(matbase)*det(matbase_prime)

Lambda = Diagonal(eigens)
poly_op1 = det(eye(m) + Lambda*Diag)
p1 = poly_operator_poly(poly_op1,p1,a)
p1 = p1(y[1]=>0, y[2]=>0, y[3]=>0, z[1]=>0, z[2]=>0, z[3] => 0) 
p1 = mpRound(p1, tol2, 10)
p1_roots = mpRoots(p1)


# now we create the polynomial the non diagonalized operator acts on 
matbase2 = x*eye(n)
for i = 1:m
   matbase2 += z[i]*V[:,i]*V[:,i]'
end

matbase2_prime = x*eye(n)
for i = 1:m
   matbase2_prime += y[i]*V[:,i]*V[:,i]'
end

p2 = det(matbase2)*det(matbase2_prime)


# next we create the polynomial from which we will get the operator
op_base = eye(m) + A*Diag
poly_op = det(op_base)


p2 = poly_operator_poly(poly_op,p2,a)
p2 = p2(y[1]=>0, y[2]=>0, y[3]=>0, z[1]=>0, z[2]=>0, z[3] => 0) 
p2 = mpRound(p2, tol2, 10)
p2_roots= mpRoots(p2)


@show p2_roots
@show p1_roots

p2_roots = ComplexF64[-58.939243245960434 + 0.0im, -5.912078893494602 + 0.0im, 5.912078893494604 + 0.0im, 58.93924324596038 + 0.0im]
p1_roots = ComplexF64[-122.73094653015275 + 0.0im, -3.9487406551445816 + 0.0im, 3.948740655144583 + 0.0im, 122.73094653015252 + 0.0im]


4-element Vector{ComplexF64}:
 -122.73094653015275 + 0.0im
 -3.9487406551445816 + 0.0im
   3.948740655144583 + 0.0im
  122.73094653015252 + 0.0im

### Final Poly is not real stable<a name="realstable2"></a>

In [23]:
m = 3
n= 4
tol = 1e-8

@polyvar x z[1:3] y[1:3]

iter = 0
conj = true 
while conj && iter < 1000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial that will act as our operator
    a = [y[1] - z[1] - y[1]*z[1], y[2] - z[2] - y[2]*z[2], y[3] - z[3] - y[3]*z[3]]
    b = [y[1], y[2], y[3], z[1], z[2], z[3]]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C
    operator_poly = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    
    input_poly = x*eye(n)
    for i=1:m
        input_poly += z[i]*V[:,i]*V[:,i]'
    end
    
    input_poly2 = x*eye(n)
    for i=1:m
        input_poly2 += y[i]*V[:,i]*V[:,i]'
    end
    
    
    poly = det(input_poly)*det(input_poly2)
    final_poly = poly_operator_poly(operator_poly,poly,b)
    
    c1 = rand() + rand()*im
    output = final_poly(z[1]=>c1, z[2]=>0, z[3]=> 0, y[1]=>0, y[2]=>0, y[3]=> 0)

    if maximum(imag.(mpRoots(output))) > tol
        @show mpRoots(output)
        conj = false 
    end
    iter +=1
end
@show conj
@show iter

mpRoots(output) = ComplexF64[-1.4953419520760194 + 0.0019949115552068478im, -0.5251726061116293 - 0.12019471941189153im, -0.03826068986485442 - 0.0008717979347752064im, -1.3677050195305186e-16 - 2.9904878315629024e-16im, 0.00026491713342437906 - 0.0008714594350504469im, 0.03876195774769532 - 0.0008064824837276173im, 0.9108000673106715 - 0.060106794622315815im, 0.0 + 0.0im]
conj = false
iter = 2


2

### Equvialence of real rootedness<a name="equiv"></a>

We want to test if the following is true

$$ \prod_{i=1}^n ( 1 + \lambda_i(A)(op_i - 1))\bigg|_{z=0} \text{ is stability preserving }  \iff \det( I + A \cdot diag(op) - A) \det(xI + \sum_i z_i v_i v_i^\top)^2 \bigg|_{z=0} \text{ is real stable }$$

Where $\lambda_i(A)$ refers to the eigenvalues of A and op refers to a vector of differential operator (e.g $[1 + \partial_{z_1}, 1 + \partial_{z_2}, 1 + \partial_{z_3}]$)

we give only part of the experiments we did, in total we looked at the operators:

\begin{align*}
    &op = 1 + \partial_y  \\
    &op = 1 - \partial_y \\
    &op = 1 - \partial_y^2 \\
    &op = 1 + \partial_y^2 \\
    &op = 1 + \partial_y - \partial_z - \partial_y \partial_z \\
    &op = 1 + \partial_y + \partial_z + \partial_y \partial_z \\
    &op = 1 -\partial_z + \partial_z^2 \\
\end{align*}

and for all of the received either real or complex roots, following the above equivalence exactly

1) op = $1 + \partial_y$

here we expect real roots

In [24]:
m = 4
n=4
tol = 1e-8

@polyvar x z[1:4] 

iter = 0
conj = true 
while conj && iter < 1000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial that will act as our operator
    a = [z[1], z[2], z[3], z[4]]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C
    operator_poly = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    input_poly = x*eye(n)

    for i=1:m
        input_poly += z[i]*V[:,i]*V[:,i]'
    end
    poly = det(input_poly)
    final_poly = poly_operator_poly(operator_poly,poly,a)
    
    output = final_poly(z[1]=>0, z[2]=>0, z[3]=> 0, z[4]=> 0)

    if maximum(abs.(imag.(mpRoots(output)))) > tol
        @show mpRoots(output)
        conj = false 
    end
    iter +=1
end
@show conj
@show iter




conj = true
iter = 1000


1000

op = $ 1 - \partial_y^2$

here we expect real roots

In [25]:
m = 4
n=3
tol = 1e-8

@polyvar x z[1:4] 

iter = 0
conj = true 
while conj && iter < 1000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial that will act as our operator
    a = [-z[1]^2, -z[2]^2, -z[3]^2, -z[4]^2]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C
    operator_poly = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    input_poly = x*eye(n)

    for i=1:m
        input_poly += z[i]*V[:,i]*V[:,i]'
    end

    poly = det(input_poly)*det(input_poly)

    final_poly = poly_operator_poly(operator_poly,poly,[z[1], z[2], z[3], z[4]])
    
    output = final_poly(z[1]=>0, z[2]=>0, z[3]=>0, z[4]=>0)

    if maximum(abs.(imag.(mpRoots(output)))) > tol
        @show mpRoots(output)
        conj = false 
    end
    iter +=1
end
@show conj
@show iter






conj = true
iter = 1000


1000

op = $ 1 + \partial_y^2$

here we expect complex roots

In [26]:
m = 4
n=2
tol = 1e-8

@polyvar x z[1:4] 

iter = 0
conj = true 
while conj && iter < 10000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial that will act as our operator
    a = [z[1]^2, z[2]^2, z[3]^2, z[4]^2]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C
    operator_poly = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    input_poly = x*eye(n)

    for i=1:m
        input_poly += z[i]*V[:,i]*V[:,i]'
    end

    poly = det(input_poly)*det(input_poly)

    final_poly = poly_operator_poly(operator_poly,poly,[z[1], z[2], z[3], z[4]])
    
    output = final_poly(z[1]=>0, z[2]=>0, z[3]=>0, z[4]=>0)

    if maximum(abs.(imag.(mpRoots(output)))) > tol
        @show mpRoots(output)
        conj = false 
    end
    iter +=1
end
@show conj
@show iter

mpRoots(output) = ComplexF64[-1.1102230246251565e-16 + 0.7929453047142051im, -8.326672684688674e-17 - 0.7929453047142055im, -4.1550155686916256e-20 + 0.14206188706478376im, 5.485684667323074e-18 - 0.14206188706478373im]
conj = false
iter = 1


1

## Third operator representation <a name="op3"></a>

$$\prod_{i=1}^n(1 -\partial_{z_i}\partial_{w_i}) ( 1 + \partial_{y_i} \partial_{w_i})(1 - \partial_{z_i} \partial_{y_i} \partial_{w_i})\det(x I  + \sum_i z_i v_i v_i^\top)\det(xI+ \sum_i y_i v_i v_i^\top)g_{\mu}(\mathbf{1} + \mathbf{w})$$

In [27]:
function op3(p, i)
    term = mvp.differentiate(p,w[i],1)
    term1 = mvp.differentiate(term,z[i],1)
    term2 = mvp.differentiate(term,y[i],1)
    term3 = mvp.differentiate(term2,z[i],1)
    output = (p - term1 + term2 - term3)
    return output
end

op3 (generic function with 1 method)

first we test what happens if we set all variables to the same variable instead of 0 to obtain a univariate polynomial

In [32]:
m = 3
n=2
tol = 1e-8

@polyvar x z[1:3] y[1:3] w[1:3]

iter = 0
conj = true 
while conj && iter < 1000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial g
    a = [w[1], w[2], w[3]]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C - A
    g = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    input_poly = x*eye(n)
    for i=1:m
        input_poly += y[i]*V[:,i]*V[:,i]'
    end
    
    input_poly2 = x*eye(n)
    for i=1:m
        input_poly2 += z[i]*V[:,i]*V[:,i]'
    end
    
    p3 = det(input_poly)*det(input_poly2)*g
    
    c1 = rand() + rand()*im
    c2 = rand() + rand()*im
    c3 = rand() + rand()*im
    c4 = rand() + rand()*im
    c5 = rand() + rand()*im 
    c6 = rand() + rand()*im
    c7 = rand() + rand()*im
    c8 = rand() + rand()*im
    c9 = rand() + rand()*im
    for i = 1:3
        p3 = op3(p3, i)(z[1]=>x, z[2]=>x, z[3]=>x, y[1]=>x, y[2]=>x, y[3]=>x, w[1]=>x, w[2]=>x, w[3]=>x)
    end
    
    for elm in mpRoots(p3)
        if imag(elm) > tol && real(elm) > tol
            @show elm
            @show mpRoots(p3)
            conj = false 
        end
    end
    iter +=1
end
@show conj
@show iter

conj = true
iter = 1000


1000

next we see what happens when we randomly set the variables to elements in the open 1 quadrant except for one

In [ ]:
m = 3
n=2
tol = 1e-8

@polyvar x z[1:3] y[1:3] w[1:3]

iter = 0
conj = true 
while conj && iter < 1000

    # first we generate A
    A = randn(m, m)
    B = A*A'
    d = maximum(abs.(eigvals(B)))
    A = B /d

    # then we generate the polynomial g
    a = [w[1], w[2], w[3]]
    C = Diagonal(a)
    matbase_g = eye(m) + A*C - A
    g = det(matbase_g)

    # next we generate a polynomial to act on
    V = rand(n,m)
    input_poly = x*eye(n)
    for i=1:m
        input_poly += y[i]*V[:,i]*V[:,i]'
    end
    
    input_poly2 = x*eye(n)
    for i=1:m
        input_poly2 += z[i]*V[:,i]*V[:,i]'
    end
    
    p3 = det(input_poly)*det(input_poly2)*g
    
    for i = 1:3
        p3 = op3(p3, i)
    end
    
    iter2 = 0
    while iter2 < 1000
        c1 = rand() + rand()*im
        c2 = rand() + rand()*im
        c3 = rand() + rand()*im
        c4 = rand() + rand()*im
        c5 = rand() + rand()*im 
        c6 = rand() + rand()*im
        c7 = rand() + rand()*im
        c8 = rand() + rand()*im
        c9 = rand() + rand()*im
        
        p3_test = p3(z[1]=>c1, z[2]=>c2, z[3]=>c3, y[1]=>c4, y[2]=>c5, y[3]=> c6, w[1]=>c7, w[2]=>c8, w[3]=> c9)

        for elm in mpRoots(p3_test)
            if imag(elm) > tol && real(elm) > tol
                @show elm
                @show mpRoots(p3_test)
                conj = false 
            end
        end
    end
    iter +=1
end
@show conj
@show iter